In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

In [2]:
data_frame = pd.read_csv("./Dataset/Data.csv")

In [3]:
data_frame

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18393,5/28/99 23:58,0,-0.877441,0.786430,0.406426,135.301215,0.112295,26.300392,-0.159185,0.058823,...,-0.773514,-0.752385,2.682413,6.944644,-37.795661,-0.860218,0.010220,0.895685,-0.011242,0
18394,5/29/99 0:00,0,-0.843988,0.633086,0.561918,133.228949,0.141332,25.678597,-0.159185,0.058823,...,-0.773514,-0.752385,2.683338,0.507755,-39.357199,-0.915698,0.010620,0.175348,-0.011235,0
18395,5/29/99 0:02,0,-0.826547,0.450126,0.334582,134.977973,0.170370,25.056801,-0.159185,0.048752,...,-0.773514,-0.752385,2.684263,2.809146,-39.357199,-1.409596,0.013323,0.621020,-0.011228,0
18396,5/29/99 0:04,0,-0.822843,0.419383,0.387263,135.658942,0.199422,24.435005,-0.159185,0.048752,...,-0.773514,-0.752385,2.685189,2.164859,-39.357199,-0.860218,0.012888,1.390902,-0.011221,0


In [4]:
target_time = '5/21/99 23:58'
training_cutoff = data_frame[data_frame['time'] == target_time]
training_cutoff

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
13616,5/21/99 23:58,0,-0.184264,5.963519,2.541319,-24.871759,1.01531,-64.013679,-0.089298,0.038986,...,5.012619,10.052303,2.654354,1.240543,-150.29198,0.329967,-0.002612,0.136225,0.000434,0


In [5]:
training_last_index = training_cutoff.index[0]
training_last_index

13616

In [6]:
cat_att1 =data_frame["x28"].values
cat_att2 =data_frame["x61"].values

In [7]:
target = data_frame["y"]
features = data_frame.drop(columns=["y", "time","x28","x61"])

In [8]:
target

0        0
1        0
2        0
3        0
4        0
        ..
18393    0
18394    0
18395    0
18396    0
18397    0
Name: y, Length: 18398, dtype: int64

In [9]:
features

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60
0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,-0.059966,-0.038189,...,29.984624,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076
1,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,-0.059966,-0.038189,...,29.984624,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083
2,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,-0.030057,-0.018352,...,29.984624,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090
3,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,-0.019986,-0.008280,...,29.984624,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097
4,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,-0.030057,-0.008280,...,29.984624,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18393,-0.877441,0.786430,0.406426,135.301215,0.112295,26.300392,-0.159185,0.058823,-0.080108,-0.038189,...,29.984624,-0.773514,-0.752385,2.682413,6.944644,-37.795661,-0.860218,0.010220,0.895685,-0.011242
18394,-0.843988,0.633086,0.561918,133.228949,0.141332,25.678597,-0.159185,0.058823,-0.080108,-0.038189,...,29.984624,-0.773514,-0.752385,2.683338,0.507755,-39.357199,-0.915698,0.010620,0.175348,-0.011235
18395,-0.826547,0.450126,0.334582,134.977973,0.170370,25.056801,-0.159185,0.048752,-0.080108,-0.038189,...,29.984624,-0.773514,-0.752385,2.684263,2.809146,-39.357199,-1.409596,0.013323,0.621020,-0.011228
18396,-0.822843,0.419383,0.387263,135.658942,0.199422,24.435005,-0.159185,0.048752,-0.080108,-0.038189,...,29.984624,-0.773514,-0.752385,2.685189,2.164859,-39.357199,-0.860218,0.012888,1.390902,-0.011221


In [10]:
features.shape, target.shape

((18398, 59), (18398,))

In [11]:
np_targets = target.values
np_features = features.values

In [12]:
mean_values = np.mean(np_features, axis=0)
std_dev = np.std(np_features, axis=0)
standardized_features = (np_features - mean_values) / std_dev

In [13]:
np_features.shape,np_targets.shape

((18398, 59), (18398,))

In [14]:
scaler = StandardScaler()
standardized_features = scaler.fit_transform(np_features)

In [15]:
pca = PCA()
pca.fit(standardized_features)

PCA()

In [16]:
pca.components_.shape

(59, 59)

In [17]:
pca.explained_variance_,pca.explained_variance_.shape

(array([1.01310690e+01, 9.34813516e+00, 4.60365938e+00, 4.01064229e+00,
        3.16815407e+00, 2.94957557e+00, 2.41860610e+00, 2.03390639e+00,
        1.69248651e+00, 1.41400772e+00, 1.12483650e+00, 1.08005278e+00,
        1.04272615e+00, 1.00472478e+00, 9.83188723e-01, 9.40384306e-01,
        8.41534240e-01, 8.14665874e-01, 6.98403019e-01, 6.83795011e-01,
        6.37494624e-01, 5.97730473e-01, 5.84119249e-01, 5.26469487e-01,
        4.78787426e-01, 4.25099854e-01, 4.16455980e-01, 3.78068181e-01,
        3.55018106e-01, 3.29547605e-01, 3.17739544e-01, 2.97001368e-01,
        2.68001568e-01, 2.62475796e-01, 2.12808704e-01, 2.01894174e-01,
        1.90033982e-01, 1.69088598e-01, 1.58886150e-01, 1.36391654e-01,
        1.29274091e-01, 1.15258457e-01, 1.10335190e-01, 1.03161912e-01,
        9.48272947e-02, 8.39155625e-02, 7.11518235e-02, 6.77854424e-02,
        5.52953674e-02, 5.12759197e-02, 4.74703687e-02, 4.51643836e-02,
        3.80987678e-02, 3.45225044e-02, 1.30031098e-02, 9.636696

In [18]:
# Components Selection with 95% variance
k = 0
total_sum = sum(pca.explained_variance_)
current_sum = 0
while current_sum/total_sum < 0.95:
    current_sum += pca.explained_variance_[k]
    k = k+1
k

32

In [19]:
pca = PCA(n_components=k,whiten=True)
tranformed_features = pca.fit_transform(standardized_features)

In [20]:
tranformed_features.shape, standardized_features.shape

((18398, 32), (18398, 59))

In [21]:
# Early Dectection
for i in range(len(np_targets)-1):
    np_targets[i] = np_targets[i+1]
np_targets[len(np_targets)-1] =0

In [22]:
X_train = tranformed_features[0:training_last_index+1]
Y_train = np_targets[0:training_last_index+1]

X_test= tranformed_features[training_last_index+1:]
Y_test = np_targets[training_last_index+1:]

In [23]:
X_train.shape, Y_train.shape

((13617, 32), (13617,))

In [24]:
X_test.shape, Y_test.shape

((4781, 32), (4781,))

In [25]:
encoder = OneHotEncoder()

# Fit and transform the first categorical attribute
cat_att1 =encoder.fit_transform(cat_att1.reshape(-1, 1)).toarray().tolist()

# Fit and transform the second categorical attribute
cat_att2 = np.array(encoder.fit_transform(cat_att2.reshape(-1, 1)).toarray().tolist())

In [26]:
cat_att1_train = cat_att1[0:training_last_index+1]
cat_att1_test = cat_att1[training_last_index+1:]

In [27]:
cat_att2_train = cat_att2[0:training_last_index+1]
cat_att2_test = cat_att2[training_last_index+1:]

In [28]:
cat_att2_train.shape

(13617, 2)

In [29]:
X_train = np.hstack((X_train, cat_att1_train, cat_att2_train))
X_train.shape

(13617, 42)

In [30]:
X_test = np.hstack((X_test, cat_att1_test, cat_att2_test))
X_test.shape

(4781, 42)

In [31]:
X_train.shape, Y_train.shape

((13617, 42), (13617,))

In [32]:
X_test.shape, Y_test.shape

((4781, 42), (4781,))

In [138]:
svm_classifier = SVC(kernel="rbf", C=1.8,class_weight='balanced', gamma=0.007)
"""grid = {
    'C': np.logspace(-3, 3, 7),
    'gamma': np.logspace(-3, 3, 7)
}
grid_search_clf = GridSearchCV(svm_classifier, grid, scoring='f1')"""

"grid = {\n    'C': np.logspace(-3, 3, 7),\n    'gamma': np.logspace(-3, 3, 7)\n}\ngrid_search_clf = GridSearchCV(svm_classifier, grid, scoring='f1')"

C=1.5, gamma = 0.007 -> 0.03163444639718805
C=1.7, gamma = 0.007 -> 0.0352112676056338
C=1.8, gamma = 0.007 -> 0.035398230088495575

In [34]:
#grid_search_clf.fit(X_train,Y_train)

In [35]:
#grid_search_clf.best_score_

In [36]:
#grid_search_clf.best_estimator_

In [139]:
svm_classifier.fit(X_train,Y_train)

SVC(C=1.8, class_weight='balanced', gamma=0.007)

In [140]:
Y_pred = svm_classifier.predict(X_test)

In [141]:
f1 = f1_score(Y_pred, Y_test)
f1

0.035398230088495575